<a href="https://colab.research.google.com/github/stefymojica/MACC/blob/main/Modelo_clinico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agente para la optimizacion de recursos en un modelo clinico
## Matematicas aplicadas y ciencias de la computacion
## Estudiante : Stefany Paola Mojica Melo

## Contextualizacion

Este agente simula una clinica privada de alta complejidad en horas de la tarde/noche, donde se concentra el mayor flujo de pacientes y la carga asistencial critica

- 1 tick corresponde 1 minutos, 20 ticks corresponde a 20 minutos y asi sucesivamente
- Hay 3 agentes:
  - pacientes
  - medicos
  - triaje
- Hay un modelo
  - Clinica donde los agentes van a interactuar
- En el triaje atienden 2 doctores y aproximadamente se demoran 10 minutos atendiendo a los pacientes - falta
- Hay pacientes de manera aleatoria que llegan en nivel 5 y los envian nuevamente a sus casas
- Los pacientes llegan y el triaje asigna el nivel de riesgo dependiendo de la situacion del paciente
- A la clinica llegan cada 1 hora  entre 15 a 22 pacientes
- Los doctores tienen un tiempo especifico que se toman para tratar a los pacientes de acuerdo a su nivel de criticidad que le haya asignado el triaje
- los pacientes llegan y el triaje les asigana un nivel de criticidad son 5 niveles.
  - nivel 1 - situacion de riesgo vital, requiere accion inmediata
    - Minutos de atencion: 30 minutos maximo
  - nivel 2 - Emergencia. requiere atencion en un tiempo maximo de 30 minutos
    - Minutos de atencion:
  - nivel 3 - Urgencia. se debe atender en un tiempo no mayor a 30 minutos
    - Minutos de atencion:
  - nivel 4 - urgencia menor. Puede esperar hasta 120 minutos
    - Minutos de atencion:
  - nivel 5 - sin urgencia. puede esperar hasta 180 minutos
    - Minutos de atencion:

## Instalar librerias

In [2]:
%pip install mesa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.4 MB/s eta 0:00:00


In [3]:
import mesa
print("Versión de Mesa instalada:", mesa.__version__)

Versión de Mesa instalada: 3.3.0


In [4]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random
import heapq

## Creacion de Agentes y Modelo

In [12]:
class SchedulerManual:
    def __init__(self, model):
        self.model = model
        self.agents = []

    def add(self, agent):
        self.agents.append(agent)

    def step(self):
        random.shuffle(self.agents)
        for agent in self.agents:
            agent.step()

class Paciente(Agent):
    def __init__(self, unique_id, model):
        super().__init__(model)
        self.unique_id = unique_id
        self.prioridad = None
        self.atendido = False

    def step(self):
        pass

class Triaje(Agent):
    def __init__(self, unique_id, model):
        super().__init__(model)
        self.unique_id = unique_id

    def step(self):
        if len(self.model.pacientes_sin_triaje) > 0:
            paciente = self.model.pacientes_sin_triaje.pop(0)
            self.asignar_prioridad(paciente)

    def asignar_prioridad(self, paciente):
        r = random.random()
        if   r < 0.05: paciente.prioridad = 1
        elif r < 0.20: paciente.prioridad = 2
        elif r < 0.50: paciente.prioridad = 3
        elif r < 0.80: paciente.prioridad = 4
        else:          paciente.prioridad = 5
        heapq.heappush(self.model.cola_prioridad_triaje, (paciente.prioridad, paciente.unique_id, paciente))

class Doctor(Agent):
    def __init__(self, unique_id, model):
        super().__init__(model)
        self.unique_id = unique_id
        self.ocupado = False
        self.paciente_actual = None
        self.tiempo_restante = 0

    def step(self):

        if self.ocupado:
            self.tiempo_restante -= 1
            if self.tiempo_restante <= 0:
                self.ocupado = False
                self.paciente_actual = None
            return

        # si hay pacientes con prioridad toma al mas urgente
        if len(self.model.cola_prioridad_triaje) > 0:
            _, _, paciente = heapq.heappop(self.model.cola_prioridad_triaje)
            self.atender(paciente)

    def atender(self, paciente):
        self.ocupado = True
        self.paciente_actual = paciente
        paciente.atendido = True
        self.model.pacientes_atendidos += 1

        tiempo_por_prioridad = {1: 20, 2: 15, 3: 12, 4: 10, 5: 10}
        self.tiempo_restante = tiempo_por_prioridad[paciente.prioridad]

class Clinica(Model):
    def __init__(self, n_doctores, semilla=None):
        super().__init__()
        if semilla is not None:
            random.seed(semilla)

        self.n_doctores = n_doctores
        self.pacientes_sin_triaje = []   # pacientes sin prioridad
        self.cola_prioridad_triaje = []  # pacientes que se les asigno prioridad
        self.pacientes_atendidos = 0
        self.tick = 0

        self.schedule = SchedulerManual(self)

        for d in range(n_doctores):
            self.schedule.add(Doctor(f"D{d}", self))


        self.triaje = Triaje("T1", self)
        self.schedule.add(self.triaje)

    def _llegada_ola(self):
        # En 1 hora, llega una ola de 15 a 22 pacientes
        if self.tick % 60 == 0:
            nuevos = random.randint(15,22)
            for i in range(nuevos):
                pid = f"P{self.tick}_{i}"
                p = Paciente(pid, self)
                self.schedule.add(p)
                self.pacientes_sin_triaje.append(p)

    def step(self):
        self._llegada_ola()
        self.schedule.step()
        self.tick += 1

    def mostrar_estado(self):
        h = self.tick // 60
        m = self.tick % 60
        print(f"\n===== MINUTO {self.tick} ({h:02d}:{m:02d}) =====")


        if len(self.cola_prioridad_triaje) == 0:
            print("Cola de espera: VACÍA")
        else:
            cola_legible = [(prio, pac.unique_id) for (prio, _, pac) in self.cola_prioridad_triaje]
            print("Cola de espera (prioridad, paciente):", cola_legible)

        for ag in self.schedule.agents:
            if isinstance(ag, Doctor):
                if ag.ocupado:
                    print(f"👨‍⚕️ Doctor {ag.unique_id} atendiendo a {ag.paciente_actual.unique_id} "
                          f"(restan {ag.tiempo_restante} min)")
                else:
                    print(f"✅ Doctor {ag.unique_id} está libre")

        #print(f"📈 Total atendidos: {self.pacientes_atendidos}")

        pacientes_faltantes = len(self.cola_prioridad_triaje) + len(self.pacientes_sin_triaje)
        print(f"📈 Total atendidos: {self.pacientes_atendidos}")
        print(f"⏳ Pacientes en espera total: {pacientes_faltantes}")

In [15]:
if __name__ == "__main__":
    clinica = Clinica(n_doctores=2, semilla=42)
    for _ in range(180):
        clinica.step()
        if clinica.tick % 30 == 0:
            clinica.mostrar_estado()


===== MINUTO 30 (00:30) =====
Cola de espera (prioridad, paciente): [(2, 'P0_6'), (2, 'P0_9'), (2, 'P0_8'), (3, 'P0_10'), (4, 'P0_7'), (4, 'P0_15'), (3, 'P0_4'), (5, 'P0_5'), (4, 'P0_11'), (5, 'P0_2'), (5, 'P0_12')]
👨‍⚕️ Doctor D0 atendiendo a P0_14 (restan 13 min)
👨‍⚕️ Doctor D1 atendiendo a P0_13 (restan 9 min)
📈 Total atendidos: 5
⏳ Pacientes en espera total: 11

===== MINUTO 60 (01:00) =====
Cola de espera (prioridad, paciente): [(3, 'P0_4'), (4, 'P0_11'), (4, 'P0_15'), (5, 'P0_2'), (4, 'P0_7'), (5, 'P0_5'), (5, 'P0_12')]
👨‍⚕️ Doctor D0 atendiendo a P0_10 (restan 12 min)
👨‍⚕️ Doctor D1 atendiendo a P0_9 (restan 11 min)
📈 Total atendidos: 9
⏳ Pacientes en espera total: 7

===== MINUTO 90 (01:30) =====
Cola de espera (prioridad, paciente): [(3, 'P60_13'), (3, 'P60_16'), (3, 'P60_5'), (3, 'P60_9'), (3, 'P60_17'), (4, 'P0_15'), (5, 'P0_12'), (4, 'P60_0'), (4, 'P0_11'), (4, 'P0_7'), (3, 'P60_4'), (5, 'P0_5'), (5, 'P60_1'), (5, 'P60_6'), (5, 'P60_7'), (5, 'P0_2'), (4, 'P60_8'), (5, 'P60